In [24]:
import numpy as np
from PIL import Image, ImageDraw
import math

In [25]:
def conv(image, filter):
    h, w = image.shape
    k_h = int(len(filter) / 2)
    k_w = int(len(filter[0]) / 2)

    # initialize image after conv
    image_conv = np.zeros((h, w))

    # padding
    padded_image = np.pad(image, ((k_h, k_h), (k_w, k_w)), mode="edge")

    # convolving
    for i in range(h):
        for j in range(w):
            image_conv[i][j] = np.einsum(
                "ij, ij -> ",
                padded_image[i : i + (2 * k_w + 1), j : j + (2 * k_h + 1)],
                filter[::-1, ::-1],
            )

    return image_conv


def edge_detection(image, sigma):
    # gaussian filter
    def gaussian_filter(k_h, k_w, sigma):
        gaussian = np.zeros((2 * k_h + 1, 2 * k_w + 1))
        for i in range(0, 2 * k_h + 1):
            for j in range(0, 2 * k_w + 1):
                gaussian[i][j] = math.exp(
                    -((k_h - i) ** 2 + (k_w - j) ** 2) / (2 * sigma**2)
                ) / (2 * math.pi * sigma**2)

        return gaussian

    # sobel filter
    def sobel_filter(mode):
        if mode == "X":
            return np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
        if mode == "Y":
            return np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

    # Non Maximal Suppresion
    def NMS(I_m, I_o):
        h, w = I_m.shape

        I_suppresed = np.zeros((h, w))
        for i in range(h):
            for j in range(w):
                magnitude = I_m[i][j]
                orientation = I_o[i][j]

                is_edge = True
                if orientation >= -1 / 8 * np.pi and orientation < 1 / 8 * np.pi:
                    if (i - 1 >= 0 and I_m[i - 1][j] > magnitude) or (
                        i + 1 <= h - 1 and I_m[i + 1][j] > magnitude
                    ):
                        is_edge = False
                elif orientation >= 1 / 8 * np.pi and orientation < 3 / 8 * np.pi:
                    if (
                        i - 1 >= 0 and j + 1 <= w - 1 and I_m[i - 1][j + 1] > magnitude
                    ) or (
                        i + 1 <= h - 1 and j - 1 >= 0 and I_m[i + 1][j - 1] > magnitude
                    ):
                        is_edge = False
                elif orientation >= -3 / 8 * np.pi and orientation < -1 / 8 * np.pi:
                    if (
                        i - 1 >= 0 and j - 1 >= 0 and I_m[i - 1][j - 1] > magnitude
                    ) or (
                        i + 1 <= h - 1
                        and j + 1 <= w - 1
                        and I_m[i + 1][j + 1] > magnitude
                    ):
                        is_edge = False
                else:
                    if (j - 1 >= 0 and I_m[i][j - 1] > magnitude) or (
                        j + 1 <= w - 1 and I_m[i][j + 1] > magnitude
                    ):
                        is_edge = False

                if is_edge:
                    I_suppresed[i][j] = I_m[i][j]
                else:
                    I_suppresed[i][j] = 0

        return I_suppresed

    # gaussian blurring
    gaussian = gaussian_filter(1, 1, sigma)
    image_blurred = conv(image, filter=gaussian)

    # gradient by X, Y
    I_x = conv(image_blurred, filter=sobel_filter(mode="X"))
    I_y = conv(image_blurred, filter=sobel_filter(mode="Y"))

    # avoid zero division
    I_y[I_y == 0] += 1e-6

    # Edge Magnitude, Orientation
    I_m = np.sqrt(I_x**2 + I_y**2)
    I_o = np.arctan(I_x / I_y)

    # Non Maximal Suppression
    I_m = NMS(I_m, I_o)

    return I_m, I_o, I_x, I_y

In [26]:
def hough_lines(image, threshold, rho_res, theta_res):
    img_array = image

    # Get image dimensions
    height, width = img_array.shape

    # Calculate maximum possible distance from origin to any line
    max_rho = int(np.ceil(np.sqrt(width**2 + height**2)))

    # Create parameter space
    rho_range = np.arange(-max_rho, max_rho + rho_res, rho_res)
    theta_range = np.arange(0, np.pi, theta_res)

    # Initialize accumulator array
    num_rho = len(rho_range)
    num_theta = len(theta_range)
    H = np.zeros((num_rho, num_theta), dtype=int)

    # Find edge pixels above threshold
    edge_pixels = np.where(img_array >= threshold)
    y_coords, x_coords = edge_pixels

    # Perform Hough voting
    for i in range(len(x_coords)):
        x, y = x_coords[i], y_coords[i]

        for theta_idx, theta in enumerate(theta_range):
            # Calculate rho using normal form equation: ρ = x·cos(θ) + y·sin(θ)
            rho = x * np.cos(theta) + y * np.sin(theta)

            # Find the closest rho bin
            rho_idx = np.abs(rho_range - rho).argmin()

            # Increment the accumulator cell
            H[rho_idx, theta_idx] += 1

    return H

In [27]:
def hough_lines_peak(H, rho_res, theta_res, num_lines):
    # Lists to store results
    line_rhos = []
    line_thetas = []

    # Create a copy of H to apply non-maximum suppression
    H_nms = H.copy()

    # Define window size for non-maximum suppression (neighborhood to suppress)
    window_size = max(3, int(min(H.shape) / 50))  # Adaptive window size

    # Calculate rho and theta values corresponding to accumulator indices
    num_rhos, num_thetas = H.shape
    max_rho = (num_rhos * rho_res) / 2  # Half the total rho range

    # Create rho and theta arrays
    rho_values = np.linspace(-max_rho, max_rho, num_rhos)
    theta_values = np.linspace(0, np.pi, num_thetas, endpoint=False)

    # Detect lines by finding peaks
    for _ in range(num_lines):
        # Find the maximum value in the accumulator
        max_idx = np.unravel_index(np.argmax(H_nms), H_nms.shape)

        if H_nms[max_idx] == 0:
            # No more peaks to find
            break

        # Convert indices to rho and theta values
        rho_idx, theta_idx = max_idx

        # Get the actual rho and theta values
        rho = rho_values[rho_idx]
        theta = theta_values[theta_idx]

        # Add to results
        line_rhos.append(rho)
        line_thetas.append(theta)

        # Apply non-maximum suppression by zeroing a window around the peak
        rho_window_start = max(0, rho_idx - window_size)
        rho_window_end = min(H_nms.shape[0], rho_idx + window_size + 1)
        theta_window_start = max(0, theta_idx - window_size)
        theta_window_end = min(H_nms.shape[1], theta_idx + window_size + 1)

        H_nms[rho_window_start:rho_window_end, theta_window_start:theta_window_end] = 0

    return line_rhos, line_thetas

In [28]:
def hough_circles(image, threshold, radius_range, a_res, b_res, r_res):
    height, width = image.shape

    # Extract radius range
    radius_min, radius_max = radius_range

    # Create parameter space
    a_range = np.arange(0, width, a_res)
    b_range = np.arange(0, height, b_res)
    r_range = np.arange(radius_min, radius_max + r_res, r_res)

    # Calculate dimensions of accumulator array
    num_a = len(a_range)
    num_b = len(b_range)
    num_r = len(r_range)

    # Initialize 3D accumulator array
    H = np.zeros((num_a, num_b, num_r), dtype=int)

    # Find edge pixels above threshold
    edge_pixels = np.where(image >= threshold)
    y_coords, x_coords = edge_pixels

    # Perform Hough voting
    for i in range(len(y_coords)):
        x, y = x_coords[i], y_coords[i]

        # For each possible radius
        for r_idx, r in enumerate(r_range):
            # Vote in a circular pattern around the edge point
            for theta in np.linspace(
                0, 2 * np.pi, 60
            ):  # Sample 60 points on the circle
                # Calculate potential center coordinates
                a = x - r * np.cos(theta)
                b = y - r * np.sin(theta)

                # Check if center is within image bounds
                if 0 <= a < width and 0 <= b < height:
                    # Find closest bin indices
                    a_idx = min(range(len(a_range)), key=lambda i: abs(a_range[i] - a))
                    b_idx = min(range(len(b_range)), key=lambda i: abs(b_range[i] - b))

                    # Increment accumulator
                    H[a_idx, b_idx, r_idx] += 1

    return H

In [29]:
def hough_circles_peak(H, radius_range, a_res, b_res, r_res, num_circles):
    circle_as = []
    circle_bs = []
    circle_rs = []

    # Create a copy of H to apply non-maximum suppression
    H_nms = H.copy()

    # Get dimensions of the accumulator array
    num_a, num_b, num_r = H.shape

    # Define window size for non-maximum suppression (neighborhood to suppress)
    # Adaptive window size based on dimensions of the accumulator
    window_size_a = max(3, int(num_a / 30))
    window_size_b = max(3, int(num_b / 30))
    window_size_r = max(2, int(num_r / 10))

    # Calculate parameter values from indices
    radius_min, radius_max = radius_range
    a_values = np.arange(0, num_a * a_res, a_res)
    b_values = np.arange(0, num_b * b_res, b_res)
    r_values = np.arange(radius_min, radius_max + r_res, r_res)

    # Detect circles by finding peaks
    for _ in range(num_circles):
        # Find the maximum value in the accumulator
        max_idx = np.unravel_index(np.argmax(H_nms), H_nms.shape)

        if H_nms[max_idx] == 0:
            # No more peaks to find
            break

        # Convert indices to parameter values
        a_idx, b_idx, r_idx = max_idx

        # Get the actual parameter values
        a = a_values[a_idx]
        b = b_values[b_idx]
        r = r_values[r_idx]

        # Add to results
        circle_as.append(a)
        circle_bs.append(b)
        circle_rs.append(r)

        # Apply non-maximum suppression by zeroing a 3D window around the peak
        a_window_start = max(0, a_idx - window_size_a)
        a_window_end = min(num_a, a_idx + window_size_a + 1)

        b_window_start = max(0, b_idx - window_size_b)
        b_window_end = min(num_b, b_idx + window_size_b + 1)

        r_window_start = max(0, r_idx - window_size_r)
        r_window_end = min(num_r, r_idx + window_size_r + 1)

        # Zero out the window
        H_nms[
            a_window_start:a_window_end,
            b_window_start:b_window_end,
            r_window_start:r_window_end,
        ] = 0

    return circle_as, circle_bs, circle_rs

In [30]:
def draw_lines(image, line_rhos, line_thetas):
    w, h = image.size
    image_with_lines = image.copy()
    image_draw = ImageDraw.Draw(image_with_lines)

    for i in range(len(line_rhos)):
        rho = line_rhos[i]
        theta = line_thetas[i]

        a = np.cos(theta)
        b = np.sin(theta)

        # Avoid zero division
        if b == 0:
            b += 1e-6

        m = -a / b
        c = rho / b

        if abs(m) < 1:
            x1 = 0
            y1 = m * x1 + c
            x2 = w - 1
            y2 = m * x2 + c

            image_draw.line([(x1, y1), (x2, y2)], fill="orange", width=3)
        else:
            y1 = 0
            x1 = (y1 - c) / m
            y2 = h - 1
            x2 = (y2 - c) / m

            image_draw.line([(x1, y1), (x2, y2)], fill="orange", width=3)

    return image_with_lines


def draw_circles(image, circle_as, circle_bs, circle_rs):
    image_with_circles = image.copy()
    image_draw = ImageDraw.Draw(image_with_circles)

    for a, b, r in zip(circle_as, circle_bs, circle_rs):
        image_draw.ellipse(
            (a - r, b - r, a + r, b + r), outline="orange", fill=None, width=3
        )

    return image_with_circles

In [31]:
######## Parameters ########
# feel free to change this
sigma = 2
threshold = 0.08
rho_res = 2.3
theta_res = math.pi / 180
num_lines = 20
######## Parameters ########

image = Image.open("./00000.png")
image_grey = np.array(image.convert("L"))
image_grey = image_grey / 255.0

edge_image, _, __, ___ = edge_detection(image_grey, sigma)
H = hough_lines(edge_image, threshold, rho_res, theta_res)
line_rhos, line_thetas = hough_lines_peak(H, rho_res, theta_res, num_lines)
image_with_lines = draw_lines(image, line_rhos, line_thetas)

image_with_lines.save("./00000_output.png")

In [32]:
######## Parameters ########
# feel free to change this
sigma = 2
threshold = 0.1
a_res = 2.0
b_res = 2.0
r_res = 4.0
radius_range = (120, 180)
num_circles = 3
######## Parameters ########

image = Image.open("./00001.png")
image_grey = np.array(image.convert("L"))
image_grey = image_grey / 255.0

edge_image, _, __, ___ = edge_detection(image_grey, sigma)
H = hough_circles(edge_image, threshold, radius_range, a_res, b_res, r_res)
circle_as, circle_bs, circle_rs = hough_circles_peak(
    H, radius_range, a_res, b_res, r_res, num_circles
)
image_with_circles = draw_circles(image, circle_as, circle_bs, circle_rs)

image_with_circles.save("./00001_output.png")

In [33]:
image = Image.open("./input.jpg")
image_grey = np.array(image.convert("L"))
image_grey = image_grey / 255.0

In [34]:
######## Parameters ########
# feel free to change this
sigma = 2
threshold = 0.08
rho_res = 2.3
theta_res = math.pi / 180
num_lines = 20
######## Parameters ########

print("drawing default lines")
edge_image, _, __, ___ = edge_detection(image_grey, sigma)
H = hough_lines(edge_image, threshold, rho_res, theta_res)
line_rhos, line_thetas = hough_lines_peak(H, rho_res, theta_res, num_lines)
image_with_lines = draw_lines(image, line_rhos, line_thetas)

image_with_lines.save("./default-line.png")

drawing default lines


In [35]:
######## Parameters ########
# feel free to change this
sigma = 2
threshold = 0.1
a_res = 2.0
b_res = 2.0
r_res = 4.0
radius_range = (120, 180)
num_circles = 3
######## Parameters ########
print("defeault circle")
H = hough_circles(edge_image, threshold, radius_range, a_res, b_res, r_res)
circle_as, circle_bs, circle_rs = hough_circles_peak(
    H, radius_range, a_res, b_res, r_res, num_circles
)
image_with_circles = draw_circles(image, circle_as, circle_bs, circle_rs)

image_with_circles.save("./default-circle.png")

defeault circle


In [36]:
######## Parameters ########
# feel free to change this
sigma = 3
threshold = 0.03
rho_res = 1
theta_res = math.pi / 180
num_lines = 20
######## Parameters ########
print("tuned line")
H = hough_lines(edge_image, threshold, rho_res, theta_res)
line_rhos, line_thetas = hough_lines_peak(H, rho_res, theta_res, num_lines)
image_with_lines = draw_lines(image, line_rhos, line_thetas)

image_with_lines.save(
    f"./tuned-line-sigma{sigma}-threshold{threshold}-rho{rho_res}-theta{theta_res}-num{num_lines}.png"
)

tuned line


In [37]:
######## Parameters ########
# feel free to change this
sigma = 2
threshold = 0.1
a_res = 2.0
b_res = 2.0
r_res = 3.0
radius_range = (80, 90)
num_circles = 1
######## Parameters ########
print("tuned circle")
H = hough_circles(edge_image, threshold, radius_range, a_res, b_res, r_res)
circle_as, circle_bs, circle_rs = hough_circles_peak(
    H, radius_range, a_res, b_res, r_res, num_circles
)
image_with_circles = draw_circles(image, circle_as, circle_bs, circle_rs)

image_with_circles.save(
    f"./tuned-circle-sigma{sigma}-threshold{threshold}-a{a_res}-b{b_res}-r{r_res}-range{radius_range[0]}-{radius_range[1]}-num{num_circles}.png"
)

tuned circle
